In [67]:
import io
import subprocess
import re
import pickle
import random

# TPTP Formatting
fof(name,formula_type,statement)
- name is the name of the formula
- formula_type is either axiom or conjecture, for our purposes.
- Axioms are facts
- Conjectures are what we want to prove/disprove


Extra TPTP notes:
- ! is the universal quantifier
- ? is the existential quantifier
- variables are denoted with capital letters
- func(C) is a function with variable C
- => is the implication operator
- & is AND
- | is OR
- ~ is NEGATION
- <=> is EQUIVALENCE
- = is EQUALITY
- != is INEQUALITY

In [68]:
class Axiom:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",axiom,"+str(self.statement)+")."

class Conjecture:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",conjecture,"+str(self.statement)+")."

In [85]:
theorum = str(Axiom("1","b"))+str(Axiom("2","a&b=>b"))+str(Axiom("3","b=>c"))+str(Conjecture("c1","c"))

print(str(Axiom("1","a")))

file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
with io.open(file_path,'w',encoding='utf-8') as f:
    f.write(theorum)

result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
output = result.stdout.decode()

if result.returncode == 0:
    #proof found
    print(output)
    pass

elif result.returncode == 1:
    #proof not found
    print(output)
    pass

else:
    #something else happened
    print(result)
    raise Exception("Something unexpected occured")

fof(1,axiom,a).
CompletedProcess(args=['/home/anmarch/source/eprover/PROVER/eprover', '--proof-object', 'f.tptp'], returncode=3, stdout=b'', stderr=b"eprover: f.tptp:1:(Column 31):(just read '=>'): Closing bracket (')') expected, but Implication/LRArrow ('=>') read \n")


Exception: Something unexpected occured

# Parse the proof output.
Todo: get rid of unnecessary info like filenames

In [70]:
parsed_result = ""
for line in output.split('\n'):
    if len(line) > 0 and line[0] == '#':
        pass
    else:
        line = line.replace(' ','')
        print(line)
        break
        

fof(2,axiom,(a=>b),file('f.tptp',2)).


# Generate expressions using:
- and &
- or |
- not ~
- implies =>
- a1,a2,a3 etc for axiom names
- c1,c2,c3 etc for conjecture names


In [86]:
operation_list = ['&','|']
variable_list1 = ['a','b','c','d']

original = ['a','b','c']
prop_names = []

for x in original:
    prop_names.append(str(x))

ops = operation_list
new = dict()
new2 = list()


for name in prop_names:
    new[str(name)]=str(name)


for i in range(1):
    for var1 in original:
        for var2 in prop_names:
            for o in ops:
                new[str(var1)+str(o)+str(var2)]=str(var1)+str(o)+str(var2)
        
    for e in new:
        original.append(str(new[e]))

for element in new:
    for name in prop_names:
        result = '('+str(new[element])+")=>"+str(name)
        new2.append(result)

print(len(new2))
with open('data.pickle','wb') as f:
    pickle.dump(new2,f)

63


In [87]:
with open('data.pickle','rb') as f:
    data = pickle.load(f)

for i in range(10):
    print(data[random.randint(0,(len(data))-1)])

(b|c)=>c
(a|a)=>c
(c|c)=>b
(b)=>a
(b&b)=>a
(c&c)=>b
(c&c)=>b
(a)=>a
(a&c)=>a
(a&b)=>c


In [93]:
s1 = str(Axiom("1",str(data[random.randint(0,(len(data))-1)]) ))
s2 = str(Axiom("2",str(data[random.randint(0,(len(data))-1)]) ))
c1 = str(Conjecture("c1",str(variable_list1[random.randint(0,(len(variable_list1))-1)])))
print(s1)
print(s2)
print(c1)

theorum =  str(s1)+str(s2)+str(c1)
with io.open(file_path,'w',encoding='utf-8') as f:
    f.write(theorum)
result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
output = result.stdout.decode()

if result.returncode == 0:
    #proof found
    print(output)
    pass

elif result.returncode == 1:
    #proof not found
    print(output)
    pass

else:
    #something else happened
    print(result.stderr.decode())
    raise Exception("Something unexpected occured")

fof(1,axiom,(b|b)=>c).
fof(2,axiom,(b&c)=>c).
fof(c1,conjecture,b).
# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_1, plain, (c|~b)).
#
#cnf(i_0_3, negated_conjecture, (~b)).

# No proof found!
# SZS status CounterSatisfiable
# SZS output start Saturation
fof(c1, conjecture, b, file('f.tptp', c1)).
fof(c_0_1, negated_conjecture, ~b, inference(fof_simplification,[status(thm)],[inference(assume_negation,[status(cth)],[c1])])).
fof(c_0_2, negated_conjecture, ~b, inference(fof_nnf,[status(thm)],[c_0_1])).
cnf(c_0_3, negated_conjecture, (~b), inference(split_conjunct,[status(thm)],[c_0_2]), ['final']).
# SZS output end Saturation

